### Combining data from OPUS for model training 

In [ ]:
"""

import pandas as pd

# Пути к файлам (замените на свои локальные пути)
ru_file = "TED2020.en-ru.ru"
en_file = "TED2020.en-ru.en"
output_file = "TED2020_translations.xlsx"

# Читаем файлы построчно
with open(ru_file, "r", encoding="utf-8") as f:
    ru_sentences = f.readlines()

with open(en_file, "r", encoding="utf-8") as f:
    en_sentences = f.readlines()

# Проверяем, что количество строк совпадает
if len(ru_sentences) != len(en_sentences):
    print("Ошибка: Файлы содержат разное количество строк!")
else:
    print(f"Загружено {len(ru_sentences)} пар предложений.")

# Создаём DataFrame
df = pd.DataFrame({"Russian": [s.strip() for s in ru_sentences], 
                   "English": [s.strip() for s in en_sentences]})

# Сохраняем в Excel
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"🎉 Данные успешно сохранены в файл {output_file}")"

"""


### Data preparing 

In [ ]:
"""

import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("TED2020_translations.xlsx")

# Split into train, validation, and test sets
train, temp = train_test_split(df, test_size=0.2, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

# Save datasets as Excel files
train.to_excel("train.xlsx", index=False, engine="openpyxl")
val.to_excel("val.xlsx", index=False, engine="openpyxl")
test.to_excel("test.xlsx", index=False, engine="openpyxl")

print(f"Data prepared and saved: train({len(train)}), val({len(val)}), test({len(test)})")"

"""

Data prepared and saved: train(312011), val(39001), test(39002)


### Cleaning the data 

In [9]:
import pandas as pd

# Load training and validation data from Excel
train_data = pd.read_excel("train.xlsx", engine="openpyxl")
val_data = pd.read_excel("val.xlsx", engine="openpyxl")

# Drop NaN values and convert everything to strings
train_data = train_data.dropna().astype(str)
val_data = val_data.dropna().astype(str)

# Save cleaned data (optional step for debugging)
train_data.to_excel("train_cleaned.xlsx", index=False, engine="openpyxl")
val_data.to_excel("val_cleaned.xlsx", index=False, engine="openpyxl")

print("Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx")

Data cleaned and saved as train_cleaned.xlsx & val_cleaned.xlsx


### Train the MarianMT Model

In [ ]:
#!pip install transformers datasets torch sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 15.0 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 56.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 74.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.7 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 70.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 81.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 85.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 72.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 69.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#!pip install transformers[torch] accelerate>=0.26.0

In [2]:
### Small dataset for fast training

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")

# Take only a subset of data (e.g., 10,000 samples)
subset_size = 10000
df = df.sample(n=subset_size, random_state=42)  # Randomly select 10,000 rows

# Split into train/validation
train, val = train_test_split(df, test_size=0.2, random_state=42)

# Save the smaller dataset
train.to_excel("train_small.xlsx", index=False, engine="openpyxl")
val.to_excel("val_small.xlsx", index=False, engine="openpyxl")

print(f"✅ Prepared smaller dataset with {len(train)} training and {len(val)} validation samples.")

✅ Prepared smaller dataset with 8000 training and 2000 validation samples.


### Second try training with small dataset

In [3]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load the smaller dataset
train_data = pd.read_excel("train_small.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_small.xlsx", engine="openpyxl")

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load pre-trained MarianMT model
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Tokenization function
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments (optimized for faster training)
training_args = Seq2SeqTrainingArguments(
    output_dir="custom_translation_model",  # ✅ Save model in a clear directory
    num_train_epochs=2,  # ✅ Reduce epochs for faster training
    per_device_train_batch_size=16,  # ✅ Increase batch size to speed up training
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=50,
    save_total_limit=1  # ✅ Keep only the latest checkpoint
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model in a clear directory
model.save_pretrained("custom_translation_model")
tokenizer.save_pretrained("custom_translation_model")

print("🎉 Training completed! Model saved in 'custom_translation_model'")


2025-03-18 19:44:58.328579: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 19:44:58.341621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742327098.356305   18486 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742327098.360866   18486 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742327098.373535   18486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.795100,0.730192
2,0.691100,0.678546


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


🎉 Training completed! Model saved in 'custom_translation_model'


### Third try, others hyperparametrs

In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Загружаем полный датасет
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

# Преобразуем в формат Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Загружаем предобученную модель MarianMT и её токенизатор
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Функция токенизации
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Применяем токенизацию ко всему датасету
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# **Оптимизированные гиперпараметры**
training_args = Seq2SeqTrainingArguments(
    output_dir="full_translation_model",  # ✅ Сохраняем модель в понятную папку
    num_train_epochs=4,  # 🔥 Увеличили количество эпох
    per_device_train_batch_size=8,  # 🔥 Уменьшили batch size для стабильности
    per_device_eval_batch_size=8,
    learning_rate=3e-5,  # 🔥 Более мягкое обучение
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,  # ✅ Храним только последнюю модель
    gradient_accumulation_steps=4,  # 🔥 Эффективное обновление весов
    fp16=True,  # ✅ Ускорение для GPU (если используется)
    logging_dir="./logs",
    logging_steps=100
)

# Создаём Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Запускаем обучение
trainer.train()

# Сохраняем улучшенную модель
model.save_pretrained("full_translation_model")
tokenizer.save_pretrained("full_translation_model")

print("Training completed! Model saved in 'full_translation_model'")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado

AttributeError: _ARRAY_API not found

RuntimeError: Failed to import transformers.models.marian.modeling_marian because of the following error (look up to see its traceback):
numpy.core.multiarray failed to import

In [ ]:
### First try of training

In [1]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset

# Load cleaned training and validation data from Excel
train_data = pd.read_excel("train_cleaned.xlsx", engine="openpyxl")
val_data = pd.read_excel("val_cleaned.xlsx", engine="openpyxl")

# Convert Pandas DataFrame to Dataset format
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

# Load the MarianMT model and tokenizer
model_name = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

# Preprocessing function for tokenization
def preprocess_function(examples):
    inputs = tokenizer(examples["Russian"], max_length=128, truncation=True, padding="max_length")
    targets = tokenizer(examples["English"], max_length=128, truncation=True, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization with batch processing
train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2
)

# Initialize Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Start training
trainer.train()

# Save trained model
model.save_pretrained(".")
tokenizer.save_pretrained(".")

print("Training completed! Model saved")

2025-03-18 16:31:51.257754: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-18 16:31:51.270838: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742315511.285192   15960 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742315511.289643   15960 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1742315511.302109   15960 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Map:   0%|          | 0/309030 [00:00<?, ? examples/s]

Map:   0%|          | 0/38613 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.443300,0.363478
2,0.367000,0.337283
3,0.293500,0.327943


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2810: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 6, 'bad_words_ids': [[62517]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


Training completed! Model saved


In [ ]:
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

# Загружаем тестовый датасет
file_path = "assembly_WER.xlsx"
df = pd.read_excel(file_path, engine="openpyxl")

# Проверяем, есть ли колонка с текстом для перевода
if "Sentence" not in df.columns:
    raise ValueError("❌ В файле нет колонки 'Sentence'. Проверьте структуру файла!")

# Загружаем первую модель
model1_name = "custom_translation_model"
tokenizer1 = MarianTokenizer.from_pretrained(model1_name)
model1 = MarianMTModel.from_pretrained(model1_name)

# Загружаем вторую модель
model2_name = "custom_translation_model_v2"
tokenizer2 = MarianTokenizer.from_pretrained(model2_name)
model2 = MarianMTModel.from_pretrained(model2_name)

# Функция для перевода
def translate(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)
    return tokenizer.decode(translated[0], skip_special_tokens=True)

# Применяем перевод
df["Translation_Model_1"] = df["Sentence"].astype(str).apply(lambda x: translate(x, model1, tokenizer1))
df["Translation_Model_2"] = df["Sentence"].astype(str).apply(lambda x: translate(x, model2, tokenizer2))

# Сохраняем результат в новый Excel-файл
output_file = "translated_assembly_WER.xlsx"
df.to_excel(output_file, index=False, engine="openpyxl")

print(f"✅ Перевод завершён! Результаты сохранены в '{output_file}'")